In [1]:
import pandas as pd
import re
import string
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
#nltk.download('stopwords')
#nltk.download('punkt')
from nltk.util import ngrams
#from google.colab import drive
from scipy.sparse import hstack
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from scipy import sparse
import os
from sklearn.dummy import DummyClassifier
import pickle


ModuleNotFoundError: No module named 'pandas.compat'

In [2]:
# train_data_file = '/home/parush/stance/Experiments/stance_mohammed/train.txt'
# test_data_file = '/home/parush/stance/Experiments/stance_mohammed/test.txt'
# TARGETS = [ 'Atheism','Climate Change is a Real Concern', 'Feminist Movement','Hillary Clinton', 'Legalization of Abortion' ]

# df = pd.read_csv(train_data_file, sep='\t')
# df1 = pd.read_csv(test_data_file, sep='\t')


In [3]:
train_data_file = '/home/parush/stance/Experiments/SomasundaranWiebe-politicalDebates/train.txt'
test_data_file = '/home/parush/stance/Experiments/SomasundaranWiebe-politicalDebates/test.txt'
TARGETS = ['god','healthcare','guns','gayRights','abortion', 'creation']
df = pd.read_csv(train_data_file, sep='\t')
df1 = pd.read_csv(test_data_file, sep='\t')


In [4]:
# train_data_file = '/home/parush/stance/Experiments/Data_MPCHI/train.txt'
# test_data_file = '/home/parush/stance/Experiments/Data_MPCHI/test.txt'
# TARGETS = ['Are E-Cigarettes safe?','Does MMR Vaccine lead to autism in children?',
#       'Does Sunlight exposure lead to skin cancer?','Does Vitamin C prevent common cold?',
#       'Should women take HRT post-menopause?']
# df = pd.read_csv(train_data_file, sep='\t')
# df1 = pd.read_csv(test_data_file, sep='\t')


In [5]:
print("The length of train data is {}".format(len(df)))
print("The length of test data is {}".format(len(df1)))


The length of train data is 4981
The length of test data is 2141


In [6]:
vectorizer = 'tfidf'   # set 'count' or 'tfidf'
analyzer = 'both'  # set 'word' or 'both' ( word and char)

In [7]:
if vectorizer == 'count':
    if analyzer == 'word':
        vectorizer = CountVectorizer(analyzer='word',ngram_range=(1,1))
    else:
        vectorizer = CountVectorizer(analyzer='word',ngram_range=(1,3))
        char_vectorizer = CountVectorizer(analyzer='char',ngram_range=(2,5))
else:
    if analyzer == 'word':
        vectorizer = TfidfVectorizer(analyzer='word',ngram_range=(1,1))
    else:
        vectorizer = TfidfVectorizer(analyzer='word',ngram_range=(1,3))
        char_vectorizer = TfidfVectorizer(analyzer='char',ngram_range=(2,5))
        
        
        
        

In [8]:
#List of FAVOR Tweets
def get_training_data_and_labels(df):
    df_train_favor = df.loc[df['Stance'] == 'FAVOR']
    df_train_favor = df_train_favor.reset_index(drop=True)
    train_favor_tweets = df_train_favor['Tweet'].tolist()
    
    # List of AGAINST Tweets
    df_train_against = df.loc[df['Stance'] == 'AGAINST']
    df_train_against = df_train_against.reset_index(drop=True)
    train_against_tweets = df_train_against['Tweet'].tolist()
    
    #Favor + Against Tweets and Labels
    train_corpus = train_favor_tweets + train_against_tweets
    train_labels = np.append(np.ones((len(train_favor_tweets))) , np.zeros((len(train_against_tweets))))
    
    
    if analyzer == 'word':
        ngram_vectorized_data = vectorizer.fit_transform(train_corpus)
        
        return ngram_vectorized_data, train_labels
    else:
        ngram_vectorized_data = vectorizer.fit_transform(train_corpus)
        char_vectorized_data = char_vectorizer.fit_transform(train_corpus)
        l = np.hstack((ngram_vectorized_data.toarray(), char_vectorized_data.toarray()))
        train_vectorized_data = sparse.csr_matrix(l)
        
        return train_vectorized_data, train_labels 


In [9]:
#preparing test_data
def get_test_data_and_labels(df1):
    df_test_favor = df1.loc[df1['Stance']=='FAVOR']
    df_test_favor = df_test_favor.reset_index(drop=True)
    test_favor_tweets = df_test_favor['Tweet'].tolist()
    print(len(test_favor_tweets))
    
    
    df_test_against = df1.loc[df1['Stance'] == 'AGAINST']
    df_test_against = df_test_against.reset_index(drop=True)
    test_against_tweets = df_test_against['Tweet'].tolist()
    print(len(test_against_tweets))
    
    
    test_corpus = test_favor_tweets + test_against_tweets
    test_labels = np.append(np.ones((len(test_favor_tweets))) , np.zeros((len(test_against_tweets))))
    
    if analyzer == 'word':
        test_ngram_vectorized_data = vectorizer.transform(test_corpus)
        
        return test_ngram_vectorized_data, test_labels
    else:
        test_ngram_vectorized_data = vectorizer.transform(test_corpus)
        test_char_vectorized_data = char_vectorizer.transform(test_corpus)
        l2 = np.hstack((test_ngram_vectorized_data.toarray(), test_char_vectorized_data.toarray()))
        test_vectorized_data = sparse.csr_matrix(l2)
        
        return test_vectorized_data,test_labels
    
    
    
    

In [10]:
# train_data_file = '/home/parush/stance/Experiments/SomasundaranWiebe-politicalDebates/train.txt'
# test_data_file = '/home/parush/stance/Experiments/SomasundaranWiebe-politicalDebates/test.txt'
# TARGETS = ['god','healthcare','guns','gayRights','abortion', 'creation']
# #df = pd.read_csv(train_data_file, sep='\t')
# df1 = pd.read_csv(test_data_file, sep='\t')


In [11]:
# train_data_file = '/home/parush/stance/Experiments/stance_mohammed/train.txt'
# test_data_file = '/home/parush/stance/Experiments/stance_mohammed/test.txt'
# TARGETS = [ 'Atheism','Climate Change is a Real Concern', 'Feminist Movement','Hillary Clinton', 'Legalization of Abortion' ]

# #df = pd.read_csv(train_data_file, sep='\t')
# df1 = pd.read_csv(test_data_file, sep='\t')


In [18]:
train_data_file = '/home/parush/stance/Experiments/Data_MPCHI/train.txt'
test_data_file = '/home/parush/stance/Experiments/Data_MPCHI/test.txt'
TARGETS = ['Are E-Cigarettes safe?','Does MMR Vaccine lead to autism in children?',
      'Does Sunlight exposure lead to skin cancer?','Does Vitamin C prevent common cold?',
      'Should women take HRT post-menopause?']
#df = pd.read_csv(train_data_file, sep='\t')
df1 = pd.read_csv(test_data_file, sep='\t')


In [ ]:
X_train, y_train =  get_training_data_and_labels(df)
X_test, y_test = get_test_data_and_labels(df1)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:

clf = pickle.load(open('/home/parush/svm_tfidf_both.sav', 'rb'))

In [ ]:
y_true, y_pred = y_test, clf.predict(X_test)
print('Report for ', classification_report(y_true, y_pred,digits=4))